In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import imutils
from pyModbusTCP.client import ModbusClient
from pyModbusTCP import utils
import keyboard

### Important points regarding Rotation Matrix
* cos(-a) = cos(a)
* sin(-a) = sin(a)

In [2]:
def robot_to_camera(robot,a):
    """Converting Robot coordinate system to Camera (Image plane) coordinate system
    -- The translation matrix with 3x1 size has all coordinates in millimeters not meters
    
    Explanation:
            We are giving object coordinates w.r.t robot and converting them to camera coordinates, so we need translation 
            matrix(4x1) is the how far the robot from camera in camera coordinate system... think in this way: we want to 
            transform the robot coordinates in camera, that needs how robot origin w.r.t camera origin.
    """
    # converting angle(in radians) to degrees
    a = (a / 180) * np.pi

    # For this usecase the camera coordiante system is 180 degree rotated in x axis, that's why we only perform Rotation w.r.t X
    R_x = np.array([[1, 0, 0], [0, np.cos(a), -np.sin(a)], [0, np.sin(a), np.cos(a)], [0,0,0]])
    # robot coordinates in camera(image plan) coordinate system, Camera(Image plane(in mm)) center is taken as the reference for these measured values
    T_c = np.array([[-500],[ -21], [540.2], [1]])   
    
    # calculating Translation matrix 
    R_Translation_C = np.concatenate((R_x, T_c), axis = 1)
    
    # finally multiplying the robot coordinates to Transformation matrix
    return np.matmul(R_Translation_C, robot)

In [3]:
def camera_to_robot(camera,a):
    """Converting Camera (Image plane) coordinate system to Robot coordinate system
    -- The translation matrix with 3x1 size has all coordinates in millimeters not meters
    
    Explanation:
            We are giving object coordinates w.r.t robot and converting them to camera coordinates, so we need translation 
            matrix(4x1) is the how far the robot from camera in camera coordinate system... think in this way: we want to 
            transform the robot coordinates in camera, that needs how robot origin w.r.t camera origin.
    """
    # setting size of camera
    camera = np.transpose(camera)
    if camera.shape[0] == 3:
        camera = np.concatenate((camera, np.array([1.0])), axis = 0)   # shape = (4,) 1D array
        
    # converting angle(in radians) to degrees
    a = (a / 180) * np.pi

    # For this usecase the camera coordiante system is 180 degree rotated in x axis, that's why we only perform Rotation w.r.t X
    R_x = np.array([[1, 0, 0], [0, np.cos(a), -np.sin(a)], [0, np.sin(a), np.cos(a)], [0,0,0]])
    # robot coordinates in camera(image plan) coordinate system, Camera(Image plane(in mm)) center is taken as the reference for these measured values
    #T_c = np.array([[-508],[ -17], [808.2], [1]])    # np.array([[-494.5],[ -19.5], [540.2], [1]])
    T_c = np.array([[-530],[ 5.5], [856.2], [1]]) 
    
    # calculating Translation matrix 
    R_Translation_C = np.concatenate((R_x, T_c), axis = 1)
    
    # To get the object coordinates in robot, we need to use inverse of the robot to camera transformation matrix
    C_Translation_R = np.linalg.inv(R_Translation_C)
    
    # finally multiplying the robot coordinates to Transformation matrix
    return np.matmul(C_Translation_R, camera)

### Camera Intrinsics and Extrinsics, Finding 3D coordinates from pixel coordinates and Vice versa

In [4]:
def pixel_coord(three_D_coord, color_frame):
    
    """ it takes 3D world coordinates and returns 2D pixel coordinates on sensor plane
    [x_3d, y_3d, z_3d] = three_D_coord
    
    x_3d = -0.22   - 
    y_3d = 
    z_3d = 0.6602
    
    all dimenstions are in meters, because the algo inside pyrealsense takes measurments in meters,
    if we give them in mm, then we will map our inputs in out of FOV plane"""
    
    profile = color_frame.get_profile().as_video_stream_profile() 
#     x,y = rs.rs2_project_point_to_pixel(profile.get_intrinsics(),[0.345,0.242,0.6602])
#     x,y = rs.rs2_project_point_to_pixel(profile.get_intrinsics(),[0.1053,-0.0495,0.6602])
    pixel = rs.rs2_project_point_to_pixel(profile.get_intrinsics(),three_D_coord)
    
    return pixel


def three_D_coord(pixel, Depth, color_frame):
    
    """ it takes pixel coordinates and returns 3D coordinates in the relation of Depth information
    [x_p, y_p] = pixel
    Depth = 0.6602 with current set up
    x_p = between 0-640
    y_p = between 0-480  
    
    all dimenstions are in meters, because the algo inside pyrealsense takes measurments in meters,
    if we give them in mm, then we will map our inputs in out of FOV plane"""
    profile = color_frame.get_profile().as_video_stream_profile() 

#     D_coordinates = rs.rs2_deproject_pixel_to_point(profile.get_intrinsics(),[0,480], 0.6602)
    D_coordinates = rs.rs2_deproject_pixel_to_point(profile.get_intrinsics(),pixel, Depth)
    D_coordinates = np.array(D_coordinates)
    
    # To transfer coordinates to PLC, we need to transfer them from meters to millimeters
    D_coordinates = D_coordinates*1000
    
    return D_coordinates

### Method for saving all 3D contour coordinates 

In [5]:
def pixel_to_3D(contours, color_frame):
    world = []
    profile = color_frame.get_profile().as_video_stream_profile() 
    for each in contours:
        D_coordinates = three_D_coord(each,0.6962, color_frame)
        D_coordinates = camera_to_robot(D_coordinates,180)
        world.append(D_coordinates[0:3])

    return world

### ModbusConnection

In [6]:
def modbus():
    client_slave = ModbusClient(host = "194.94.86.6", port = 502, auto_open = True)
    client_slave.open()
    if client_slave.is_open():
        print("connected")
    else:
        print("not connected, try again")
    return client_slave
        
def write_registers(list, Client):
    for each,s_address in zip(list,range(len(list))):
        for value,address in zip(each, range(len(each))):
            holding_register_value = int(value*10)
            address = address+(s_address*3)
            Client.write_single_register(24576+address, utils.get_2comp(holding_register_value))
                       
#     print(Client.read_holding_registers(0,17), "Hodling_Registers")  # in modbus library the address must be an variable, indirect variables are not taken
        
def close_modbus(Client):
    Client.close()

### Camera Streaming and Contour Detection

In [44]:

def Camera_Module():
    
    # Configure depth and color streams
    pipeline = rs.pipeline()
    config = rs.config()

    # Get device product line for setting a supporting resolution
    pipeline_wrapper = rs.pipeline_wrapper(pipeline)
    pipeline_profile = config.resolve(pipeline_wrapper)
    device = pipeline_profile.get_device()
    device_product_line = str(device.get_info(rs.camera_info.product_line))

    found_rgb = False
    for s in device.sensors:
        if s.get_info(rs.camera_info.name) == 'RGB Camera':
            found_rgb = True
            break
    if not found_rgb:
        print("The demo requires Depth camera with Color sensor")
        exit(0)

    config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

    if device_product_line == 'L500':
        config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 30)
    else:
        config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

    # Start streaming
    pipeline.start(config)
    
    # Start Modbus Communication
    client = modbus()
    
    try:
        while True:
            
            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()
            depth_frame = frames.get_depth_frame()
            if not color_frame:
                continue

            # Convert images to numpy arrays
            color_image = np.asanyarray(color_frame.get_data())
            depth_image = np.asanyarray(depth_frame.get_data())
            color_image = cv2.flip(color_image,-1)
            
           
            
            # Modbus Homed Command
            homed = client.read_discrete_inputs(0,1)
            #print(homed[0])
            
            
            # start of Contour Detection
            conbri_image = cv2.convertScaleAbs(color_image, alpha=2.1, beta=35)
            grayimage = cv2.cvtColor(conbri_image, cv2.COLOR_BGR2GRAY)
            ret, thresh_img =  cv2.threshold(grayimage, 211, 255, cv2.THRESH_BINARY_INV)  #cv2.threshold(grayimage, 89, 255, cv2.THRESH_BINARY) oldone
            contours, hierarchy = cv2.findContours(thresh_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            
#             grayimage = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
#             ret, thresh_img = cv2.threshold(grayimage, 65, 200, cv2.THRESH_BINARY)
#             contours, hierarchy = cv2.findContours(thresh_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    #         contours = imutils.grab_contours(contours)
#             cnts_sorted = sorted(contours, key=lambda x: cv2.contourArea(x))

            list_of_contours = []
            for c in contours:
                area = cv2.contourArea(c)
    #             if area < 175 and area > 120 and x < 2:
                if area > 500 and area < 50000:
#                 if area > 9000:

                    M = cv2.moments(c)
                    # Finding centroid
                    cX = int(M["m10"]/ M["m00"])
                    cY = int(M["m01"]/ M["m00"])
                    list_of_contours.append([cX, cY])
                    cv2.drawContours(color_image, [c], -1,(0,255,0), 1)
                    cv2.circle(color_image, (cX, cY), 1, (0,255,255), -1)

            # End of Contour Detection

            # Finding pixel coordinates from 3D coordinates( in meters)
            x,y = pixel_coord([0.1053,-0.0495,0.6962], color_frame)
            cv2.circle(color_image, [int(x),int(y)], 1, (0,0,255), -1)
            # Drawing center at the principle point 
            cv2.circle(color_image, [321,232], 1, (255,0,255), -1)
#             cv2.circle(color_image, [320,240], 1, (0,255,255), -1)

            # Finding 3D world coordinates from pixel coordinates
#             D_coordinates = three_D_coord([0,240], 0.6602, color_frame)  # These values in millimeters
            _3_D_coord = pixel_to_3D(list_of_contours, color_frame)

            #cv2.imshow('Contour Detection', color_image)
             # Displaying image
            cv2.imshow('RealSense', color_image)
            cv2.imshow("const", conbri_image)
            cv2.imshow("thresh", thresh_img)
            
            # Breaking the Streaming    
            key = cv2.waitKey(1)
            if key == 27:
                cv2.destroyAllWindows()
                break
            
            if homed[0]:
                # Writing back the Modbus Values
                write_registers(_3_D_coord, client)
            
            else:
                continue
             
            #This is for safe side, it can be used later for breaking the while loop
#             if keyboard.is_pressed("q"):
#                 break

        # Destroying all windows after disrupting streaming
        cv2.destroyAllWindows()
        
        # Closing Modbus Connection
        close_modbus(client)
        
        
        # For debugging
        print(x,y)
        print(len(_3_D_coord)*3, "total number of holding registers are written")
        print(_3_D_coord)
       
        

    #finally:
    except KeyboardInterrupt:
        # Stop streaming
        pipeline.stop()
        pass

In [45]:
def main():
    Camera_Module()
    
    
if __name__ == "__main__":
    main()

connected
412.14501953125 189.3399658203125
3 total number of holding registers are written
[array([459.03064191,  23.16746305, 159.99998684])]
